# 0. 사이킷런의 기반 프레임워크

### 사이킷런 클래스
    - fit()
    - predict() 
1. Estimator 클래스
    1.1. 분류 : classifier
    1.2. 회귀 알고리즘 : Regressor 
2. Evaluation 함수 : Estimator를 일자로 받고, fit, predict를 통해 평가, 하이퍼파라미터 튜닝 등 
    2.1. cross_val_score() 
    2.2. GridSearchCV 등 
    
        

# 1. sklearn.model_selection 모듈

## 1.1. 학습/데이터 세트 분리 : train_test_split() 
from sklearn.model_selection import train_test_split
- 인자 
    - 첫 번째 파라미터 : feature data set 
    - 두 번째 파라미터 : 종속 데이터 세트 

    - test_size : 테스트 세트 크기 비율 설정
    - shuffle : 데이터를 분리하기 전에 데이터를 미리 섞을 지 결정 
    - random_state = n : 
        - 랜덤 시드와 같은 개념 
        - 설정하지 않을 경우, train_test_split() 호출시마다 무작위 분리
        - 일정한 숫자를 지정하여 동일한 데이터 세트가 나오도록 설정 

###### 학습 / 데이터 세트 분리하고 학습 

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

iris = load_iris()
dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

X_iris = iris_data['data']
y_iris = iris_data['target']

In [2]:
print(X_iris[:10]) # 종속 데이터 
print(y_iris[:10]) # 타겟 데이터 

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
[0 0 0 0 0 0 0 0 0 0]


In [3]:
# 학습용 데이터, 테스트 데이터 분리
# 테스트 데이터 비율 = 0.5, Seed = 121 
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, 
                                                    test_size=0.5, random_state=121)

dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9467


######  학습 / 데이터 세트 분리하지 않고 학습 
- 예측 결과가 100% 정확한 이유는 이미 학습한학습 데이터 세트를 기반으로 예측

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

예측 정확도: 1.0


## 1.2. 교차 검증 ( KFold, StratifiedKFold )

from sklearn.model_selection import KFold / StratifiedKFold
- 과적합(Overfitting) : 모델이 학습데이터에 과도하게 최적화되어, 실제 예측 데이터를 사용 시, 학습 데이터의 노이즈, 아웃라이어 등에 의해 성능이 매우 떨어지는 것 
- 교차 검증 
    - "여러 개"의 학습 데이터와, 테스터 데이터 세트로 나누어 학습, 평가 수행 
    - 각 세트의 수행 평가 결과에 따라, 하이퍼 파라미터 튜닝 등의 모델 최적화 더 쉽게 가능 
<img src = './img/교차검증.jpg' width = '70%' height = '70%' align = 'left' >

### 1.2.1 K-Fold Cross Validation
- K = 5인 경우 
    - 첫 번째 반복에서는처음부터 4개 등분을학습데이터 세트, 마지막5번째 등분하나를검증데이터 세트로설정하고학습데이터 세트에 서 학습수행 검증데이터 세트에서 평가를 수행
    - 5개(K개)의 예측 평가를 구했으면 이를 평균해서 K 폴드 평가 결과
<img src = './img/kfold.jpg' width = '70%' height = '70%' align = 'left' >

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])

붓꽃 데이터 세트 크기: 150


In [6]:
n_iter = 0

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환  
for train_index, test_index  in kfold.split(features):
    
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
        # 각 fold별, 학습용 세트의 feature, 검증 세트의 feature 
    y_train, y_test = label[train_index], label[test_index]
        # 각 fold별, 학습용 세트의 종속 변수, 검증 세트의 종속 변수
    
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)
        # 학습용 feature와 종속변수를 이용 학습 
    pred = dt_clf.predict(X_test)
        # decision tree classifier를 이용하여, 예측 수행 
    
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    # 테스트 셋의 실제값과, 예측값 비교 
    # accuracy_score(accuracy_score(검증 데이터의 실제값, 예측된 값))
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


### 1.2.2 Stratified K  폴드
- 분포가 고르지 못한 데이터를 학습/검증 데이터로 K개 분할 할 경우, 모든 데이터 세트가 동일한 분포를 갖도록 데이터를 분할 
- 한쪽으로 편향된 데이터를 갖고 있는 데이터를 학습시킬 경우,
    - K-Fold를 이용할 경우, 랜덤하게 학습, 검증 세트를 선택하더라도, 특정 feature들의 특성이 제대로 반영이 안될 수 있음.
- 일반적인 분류모델에서는 Stratified K Fold 사용하여야 함 


###### 편향된(Skewed, Imbalanced) 데이터 Staratified K-Fold 

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns 
import matplotlib.pyplot as plt

dt_clf = DecisionTreeClassifier(random_state = 10)
skf = StratifiedKFold(n_splits=3)
n_iter=0
k_mean = []

In [8]:
features = iris['data']
label = iris['target']

In [9]:
for train_index, test_index in skf.split(features, label):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    
    
    print('========== 교차 검증: {0} =========='.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    
    
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    k_mean.append(accuracy)
    
    print('{0} ➤ 교차 검증 정확도 : {1}'.format(n_iter, accuracy))
    
    k = int('31' + str(n_iter))
    plt.subplot(k)
    sns.distplot(features[train_index])
    
print('Strtified K-Fold 평균 정확도 :', np.mean(k_mean))

NameError: name 'iris_df' is not defined

###### 편향된(Skewed, Imbalanced) 데이터에서의 K-Fold

In [ ]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()


kfold = KFold(n_splits=3)
# kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('==========  교차 검증: {0} =========='.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    k = int('31' + str(n_iter))
    plt.subplot(k)
    sns.distplot(features[train_index])

In [ ]:
plt.subplot(211)
sns.distplot(iris_df)
plt.subplot(212)
sns.distplot(iris_df)

## 1.3. 교차 검증을 보다 간편하게 ( cross_val_score ) 
from sklearn.model_selection import cross_val_scroe, cross_validate
- Estimator를 학습(fit), 예측(predict), 평가(evaluation)시켜주므로 간단하게 교차 검증을 수행

1. cross_val_score(estimator, X, y, scoring, cv)
    - estimator 
        - 분류: Classifier
        - 회귀: Regressor 
    - X : 피처 데이터 세트
    - y : 검증 데이터 세트 
    - scoring : 예측 성능 평가 지표 
    - cv : Fold 수 
2. 반환값 : scroing 파라미터에 따른 성능 지표 측정값
3. 분류 방법 
    - Regressor : K-Fold 방식 
    - Classifier : Stratified K-Fold 방식

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 세트(cv)는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### 교차 검증과 최적 하이퍼 파라미터 튜닝 GridSearchCV 
- 데이터 세트를 cross-validation을 위한 학습/테스트 세트로 자동 분할하여, 여러 조합의 하이퍼 파라미터를 적용하여 최적의 파라미터를 찾는다. 
- CV = 3 ( 3개의 세트로 나누어 교차 검증 )이라면, 개별 하이퍼 파라미터 조합마다 각 폴딩 세트를 학습/평가하여성능을 평가 
    - ex) CV = 3, 하이퍼 파라미터 조합 = 6 이라면, 각 폴딩 세트마다 6번, 총 18회의 학습/ 평가

 
    
- GridSearchCV(estimator, param_grid, scoring, cv, refit) 
    - estimator : Classifier, regressor, pipeline 등 
    - param_grid : 키:리스트 형태 딕셔너리, estimatior의 튜닝을 위해 사용될 파라미터 
    - cv : 교차 검증을 위해 사용되는 학습/테스트 세트 개수 
    - refit : True -> 최적의 하이퍼파라미터 찾으면, estimator객체를 재학습 

In [4]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_iris = iris['data']
y_iris = iris['target']

# 학습, 테스트 데이터 분리 
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, 
                                                    test_size=0.2, random_state=121)

# Deicistion Tree Classirfier ( Estimator 클래스 )
dtree = DecisionTreeClassifier()

######  하이퍼파라미터 설정

In [5]:
### parameter 들을 dictionary{ 키 : 리스트 } 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재학습 시킴.  
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
           'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


- rank_test_score : 하이퍼 파라미터 성능 순위
- split0~2 _test_score : 각각 3개의 폴딩 세트에서 하이퍼 파라미터를 테스트한 성능 수치 
- mean_test_score : split0~2_test_score 의 평균값 
- 0~5 인덱스 : 하이퍼파라미터 조합을 보여줌 

- GridSearchCV 객체의 fit( )을 수행 시, 
    - 최고 성능을 보이는 파라미터 값과 평가는 다음 변수에 저장이 됨 
        - best_params_
        - best_score_ 
    - 최고 성능을 나타내는 파라미터값으로 재학습된 estimator 객체는 다음과 같이 가져올 수 있음 
        - best_estimator_ 

In [ ]:
print('grid_dtree.best_score_ : ', grid_dtree.best_score_)
print('grid_dtree.best_params_ : ', grid_dtree.best_params_)

In [ ]:
best_estimator = grid_dtree.best_estimator_ 
pred = best_estimator.predict(X_test)
print(accuracy_score(y_test, pred))

# 2. 데이터 전처리

## 2.1. 전처리 가이드 

### 2.1.1 전반적인 흐름



###### 1. 데이터 Type, Label 등이 일관적이지 않은 경우  
    - 프로그램에서 제공하는 함수를 통해 일괄적으로 변경 (예, SQL: Cast, Python: astype())
    - 날짜 : pandas datetime으로 변환

###### 2. 데이터 Type, Label 등이 일관적이지 않은 경우  
프로그램에서 제공하는 함수를 통해 일괄적으로 변경 (예, SQL: Cast, Python: astype())

###### 3. Missing Value
    - 샘플이 많은 경우 : 삭제해도 무관
    - 샘플이 적은 경우 
        - 카테고리형인 경우 Mode로 채우거나 분류 예측 모델링 활용 (예, Logistic Regression)
        - 수치형인 경우 Mean, Median 등 대푯값으로 채우거나 실수 예측 모델링 활용 (예, Linear Regression)
    - 회귀 및 분류 예측 모델 이용 가능 

###### 4. 텍스트 처리
    - Errors, Typo 발생의 경우 : 텍스트 처리 함수 활용 (예, Python: str.replace())
    - 같은 의미를 갖는 카테고리들 또한 동일하게 맞춰줄 필요 있음 

###### 5. 이상치(outlier)  
IQR, Z-score, MAD 등 방식으로 이상치 제거

###### 6. 변수가 많은 경우(20개 이상)  
PCA 등으로 차원 축소하거나 변수 중요도 파악후 불필요 변수 제거

###### 7. 편향된 분포의 변수가 존재하는 경우  
log, sqrt 등 함수로 분포 변환
측정 단위(scale)이 차이가 클 경우

###### 8. StarndardScale or MinMaxScaler 통해 스케일링

###### 9. 하이퍼 파라미터 튜닝

###### 10. 다중공선성 데이터 확인 
    - 독립변수끼리 강한 상관관계를 가지면 다중공선성Multicollinearity이 있다고 함 
    - 독립변수끼리 종속적인 것 자체가 회귀분석에 위배되고, 수치적인 문제 야기 
    - 다중공선성이 있을 가능성이 높은 경우
        1. F검정은 통과했으나 각각의 회귀계수가 t검정을 통과하지 못하는 경우
        2. 예상하던 것과 달리 회귀계수의 부호가 반대일 정도로 괴리가 큰 경우
        3. 데이터를 추가하거나 제거할 때 기존의 회귀계수가 극심하게 많이 변하는 경우

### 2.1.2. 데이터 결측치 처리

결측값 즉 비어있는 값이 있는 상태로 모델을 만들게 될 경우 변수간의 관계가 왜곡될 수 있기 때문에 모델의 정확성이 떨어지게 됩니다. 결측치 처리는 결측치 제거, 수치형의 경우 평균이나 중앙치로 대체하거나 범주형인 경우 mode 값으로 대체, 간단한 예측 모델로 대체하는 방식이 일반적으로 이용됩니다.

###### 데이터 결측치 처리방법1 - 삭제
결측값이 발생한 모든 관측치를 삭제하거나 데이터 중 모델에 포함시킬 변수 들 중 관측값이 발생한 모든 관측치를 삭제하는 방법이 있습니다. 그러나 전체삭제 또는 부분삭제는 실제 예측에 영향을 주는 데이터일 경우 Cost에 영향을 미칠 수 있습니다.

그렇기 때문에 삭제는 결측값이 무작위로 발생한 경우에 사용합니다. 결측값이 무작위로 발생한 것이 아닌데 삭제할 경우 왜곡된 모델이 생성될 수 있습니다.

###### 데이터 결측치 처리방법2 - 다른 값으로 대체
결측값이 발생한 경우 다른 관측치의 평균, 최빈값, 중간값으로 대체할 수 있습니다. 결측 값이 발생이 다른 변수와 관계가 있는 경우 유용하짐나 그렇지 않은 경우 모델이 왜곡될 가능성이 존재합니다.

###### 데이터 결측치 처리방법3 - 예측값 삽입
결측값이 없는 관측치를 트레이닝 데이터로 사용해서 예측모델을 만드는 방법입니다. 예측하는 방법은 Regression이나 Logistic Regression을 주로 사용합니다.

### 2.1.3. 이상 데이터 처리

이상 데이터란 기존 데이터와 동떨어진 관측치로, 모델을 왜곡할 가능성이 많은 데이터들입니다.

###### 이상 데이터 처리방법1 - 단순삭제
이상데이터가 실수로 발생한 경우에는 해당 값을 삭제하면 됩니다. 예를 들어 단순 오타나 비현실적인 응답 등입니다.

###### 이상 데이터 처리방법2 - 다른 값으로 대체
    1) 표준점수로 변환 후 -3 이하 및 +3 제거 
    2) IQR 방식 
    3) 도메인 지식 이용하거나 Binning 처리하는 방식 

## 2.2. 데이터 인코딩
from sklearn.preprocessing import LabelEncoder   
from sklearn.preprocessing import OneHotEncoder 




### 2.3.1. 레이블 인코딩  => 카레고리 데이터 분석 모델 (Classifier) 적용, Regressor 적용 시 오류 많아짐 
from sklearn.preprocessing import LabelEncoder 
- 같은 문자열을 갖는 값은 같은 숫자로 라벨링 됨 
- 카데고리 피처를 코드형숫자 값으로 변환   
- 몇몇 ML 알고리즘에는 이를 적용할 경우 예측 성능이 떨어지는 경우가 발생 가능 
    - 같은 특징을 갖는 데이터임에도, 레이블링될 때 숫자의 크고 작고가 생기기에 가중치로 여겨질 수 있음
    
<img src = './img/라벨인코딩.jpg' width = '60%' height = '60%' align = 'left' >

In [ ]:
from sklearn.preprocessing import LabelEncoder 

items = [ 'TV', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('           labels : ', labels)
inversed = encoder.inverse_transform(labels)
print('inverse_transform : ', inversed)

### 2.3.2. 원-핫 인코딩 => 적용 전 라벨 인코딩 + 2차원 데이터로 변환 => regressor 모델 사용 
from sklearn.preprocessing import OneHotEncoder 
- 문자열은 인코딩 불가능 => 모두 숫자로 미리 변환시켜줘야됨 
- 피처값의 유형에 따라 새로운 피처를 추가해 고유값에 해당하는 칼럼에만 1 을 표시 하고나머지칼럼에는0을표시
<img src = './img/원핫인코딩.jpg' width = '60%' height = '60%' align = 'left' >

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder 
import pandas as pd

items = [ 'TV', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# 라벨 인코딩 : 문자열을 숫자형으로 라벨링 
l_encoder = LabelEncoder() 
l_encoder.fit(items)
labels = l_encoder.transform(items)

# 2차원 데이터로 변환 
labels = labels.reshape(-1,1)

# 원핫 인코딩 : 0,1로 변환 
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

print( '원본 : \n', items )
print( '라벨 인코딩 : \n', labels.reshape(-1).tolist() )
print( '원핫 인코딩 :') 
print(pd.DataFrame(oh_labels.toarray()))
print( '원핫 인코딩 inverse : \n', oh_encoder.inverse_transform(oh_labels).reshape(-1).tolist() )

###### 원-핫 인코딩 == pd.getdummies(df)

In [ ]:
import pandas as pd 
df = pd.DataFrame({'items' : [ 'TV', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서'] })
pd.get_dummies(df)

## 2.3. 피쳐 스케일링과 정규화 / 표준화 (1)
import sklearn.preprocessing import StandardScaler


1. 피쳐 스케일링 : 서로 다른 변수의 값 범위를일정한 수준으로 맞추는 작업

1.1. 표준화 : 피쳐 ~ N(0,1) 변환  
    $$ xi\_new = { x - mean(x) \over stdev(x)} $$

          
1.2. 정규화 : 서로 다른 피처의 크기를통일하기 위해 크기를변환해주는 개념

  
ex) A는거리를나타내는 변수로서 값이 0 ~ l00KM /  B는금액을나타내는속성 으로 값이 0 ~ 100,000,000,000원  
    이 변수들을 모두 동일한 크기 단위로 비교하기 위해 값을모두최소 0 ~ 1의 값으로 변환하는 것

$$ xi\_new = { xi - min(x) \over max(x) - min(x) }$$  



2. 사이킷런의 Normalizer ( 정규화 모듈 ) :   
선형대수에서의 정규화 개념이 적용 ➤ 개별 벡터를 모든 피처 벡터의 크기로 나누 줌 
$$ xi\_new = { xi \over \sqrt{xi^2 + yi^2 + zi^2} } $$

##### 3. 피쳐 스케일링시 유의점 
- 학습 데이터와 검증 데이터를 따로 스케일링할 경우, 스케일링 되는 비율이 맞지 않는 문제가 발생할 수 있고, 이는 올바르지 않은 예측 결과 도출
- 학습 데이터에 fit을 적용했다면, 테스트 데이터는 fit을 하지말고, 학습 데이터에 맞춰져 있는 scaler 객체를 사용해야 같은 비율로 스케일링이 됨 

- ( 아래의 예시 ) : 학습데이터에 fit 적용 후, 테스트 데이터에 새로 fit 적용, MinMaxScaler 적용
    - 학습 데이터 : 0 ~ 10 범위를 갖음 
        - scaler.fit(train_data) => 0 ~ 1 사이의 값을 갖게 됨, 결과적으로 1/10 스케일링 적용  
    - 테스트 데이터 : 0 ~ 5 범위를 갖음 
        - scaler.fit(train_data) => 0 ~ 0.5 사이의 값을 갖게 됨, 결과적으로 1/5 스케일링 적용 
- 테스트 데이터에 다시 fit( )을 적용해서는 안 되며 학습 데이 터로 이미 fit( )이 적용된 Scaler 객체를 이용해 transform( )으로 변환

#### 4. 요약
###### - 가능하다면 전체 데이터의 스케일링 변환을 적용한 뒤 학습과 테스트 데이더로 분리
###### - 테스트 데이터 변환 시에는 fit( )이나 fit_transform( )을 적용않고 학습 데이터로 이미 fit( )된 Scaler 객체를 이용해 transform( )으로 변환

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0 부터 10까지, 테스트 데이터는 0 부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler클래스의 fit(), transform()은 2차원 이상 데이터만 가능하므로 reshape(-1, 1)로 차원 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array =  np.arange(0, 6).reshape(-1, 1)

# 최소값 0, 최대값 1로 변환하는 MinMaxScaler객체 생성
scaler = MinMaxScaler()
# fit()하게 되면 train_array 데이터의 최소값이 0, 최대값이 10으로 설정.  
scaler.fit(train_array)
# 1/10 scale로 train_array 데이터 변환함. 원본 10-> 1로 변환됨.
train_scaled = scaler.transform(train_array)
 
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

# 앞에서 생성한 MinMaxScaler에 test_array를 fit()하게 되면 원본 데이터의 최소값이 0, 최대값이 5으로 설정됨 
scaler.fit(test_array)

# 1/5 scale로 test_array 데이터 변환함. 원본 5->1로 변환.  
test_scaled = scaler.transform(test_array)

# train_array 변환 출력
print('\n원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))
print('\n학습 데이터와 테스트 데이터의 스케일링이 맞지 않음\n학습 : 1 -> 0.1   //  테스트 : 5 -> 0.1')

In [ ]:
scaler = MinMaxScaler()

# train_array에 Scale fit(), 테스트 세트도 이 Scaler를 사용해서 변환해야함 !!!!!!!!
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

# test_array에 Scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform() 만으로 변환해야 함. 
test_scaled = scaler.transform(test_array)

print('\n원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))
print('\n학습 데이터와 테스트 데이터의 스케일링이 맞음\n학습 : 1 -> 0.1   //  테스트 : 5 -> 0.5')

###  2.3.1. StandardScaler : 표준화 지원, 개별 피쳐를 N(0,1)을 따르는 분포로 변환 / 반환형식 : ndarray
import sklearn.preprocessing import StandardScaler
- 사이킷런에서 데이터의 정규분포( N(0,1) )를 가정하고 구현된 모델
    - 선형 회귀 ( Linear Regression )
    - 로지스틱 회귀 ( Logistic Regression ) 
    - RBF 커널을 이용하는 SVM 


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

#transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature 들의 평균 값')
print(iris_df_scaled.mean())
print('\nfeature 들의 분산 값')
print(iris_df_scaled.var())

### 2.3.2. MinMaxScaler : 데이터가 가우시안 분포를 따르지 않을 경우 적용, 0 ~ 1 사이 범위값으로 변환 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
# MinMaxScaler 로 데이터 셋 변환. fit() 과 transform() 호출.  
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print('feature들의 최소 값')
print(iris_df_scaled.min())

print('\nfeature들의 최대 값')
print(iris_df_scaled.max())

print('\nfeature들의 평균 값')
print(iris_df_scaled.mean())

print('\nfeature들의 분산 값')
print(iris_df_scaled.var())